In [1]:
import numpy as np
import pathlib
from nilearn.decomposition import CanICA
def ICA_decomposition(func_filenames, group, i):
    fast_ica = CanICA(n_components=30,
                    memory="nilearn_cache", memory_level=2,
                    mask_strategy='whole-brain-template',
                    do_cca=False,
                    random_state=0)
    fast_ica.fit(func_filenames)
    P = pathlib.Path('/data/origami/niusha/codes/local-experiment/io/ICAs')
    ICA_s = fast_ica.components_img_
    ICA_s.to_filename(P / (f'ICAs_{group}_{i}.nii.gz'))
    return ICA_s

In [2]:
from nilearn import masking
from nilearn.image import iter_img
from nilearn.masking import apply_mask

def Means_after_masking(ICAs,DBM_maps):

    means_after_mask = np.zeros((30,DBM_maps.shape[3]))
    for i, cur_img in enumerate(iter_img(ICAs)):
        mask_img = masking.compute_epi_mask(cur_img)
        means_after_mask[i,:] = np.mean(apply_mask(DBM_maps,mask_img),axis=1)
        
    return means_after_mask

In [3]:
import pandas as pd

subject_df = pd.read_csv("/data/origami/niusha/input/subject_IDs.csv")

N = len(subject_df.ID)
ID_map = dict(zip(range(N),subject_df.ID))

Healthy_index = np.where(subject_df.PD == 0)
Healthy_subject = subject_df.ID.iloc[Healthy_index]

PD_index = np.where(subject_df.PD == 1)
PD_subject = subject_df.ID.iloc[PD_index]

In [4]:
from nilearn.image import load_img
Original_DBMs = load_img("/data/origami/niusha/input/DBM_data.nii.gz")

In [ ]:
from random import sample
from nilearn.image import index_img
from scipy.stats import ttest_ind

P_vals = np.zeros((50,30))
T_vals = np.zeros((50,30))

for j in range(20):

    PD_sample = sample((PD_index[0]).tolist(), len(PD_index[0]))
    Healthy_sample = sample((Healthy_index[0]).tolist(), len(Healthy_index[0]))

    PD_bootstrapped_cohort = index_img(Original_DBMs, PD_sample)
    Healthy_bootstrapped_cohort = index_img(Original_DBMs, Healthy_sample)

    PD_ICAs = ICA_decomposition(PD_bootstrapped_cohort, "PD", j)
    Healthy_ICAs = ICA_decomposition(Healthy_bootstrapped_cohort, "Healthy", j)

    PD_means = Means_after_masking(PD_ICAs,PD_bootstrapped_cohort)
    Healthy_means = Means_after_masking(Healthy_ICAs,Healthy_bootstrapped_cohort)

    for i in range(30):
        T_vals[j,i], P_vals[j,i] = ttest_ind(PD_means[:,i], Healthy_means[:,i])

In [9]:
P_vals_reshaped = P_vals.reshape(P_vals.shape[0], -1)
np.savetxt("./io/p_values.txt", P_vals_reshaped)

T_vals_reshaped = T_vals.reshape(T_vals.shape[0], -1)
np.savetxt("./io/t_stat.txt", T_vals_reshaped)

(20, 30)

In [ ]:
# from nilearn.plotting import plot_prob_atlas

# # Plot all ICA components together
# plot_prob_atlas(canica_components_img, title='All ICA components')

# from nilearn.image import iter_img
# from nilearn.plotting import plot_stat_map, show

# for i, cur_img in enumerate(iter_img(canica_components_img)):
#     plot_stat_map(cur_img, display_mode="z", title="IC %d" % i,
#                   cut_coords=1, colorbar=False)

In [ ]:
# from nilearn.glm import threshold_stats_img

# ICA_tr = threshold_stats_img(stat_img=ICAs, threshold=3.0)
#The output is not compatible with mask and also I dont know what is the ouput of ICA. I tought it would be z stat, but when I set thr=3 I receive warning regarding the amount